# Fetch Codebase and Models (git에서 higan, psp-encoder 파일 불러오기)

In [ ]:
# HIGAN 불러오기
import os
os.chdir('/content')
CODE_DIR = 'higan'  # HiGAN 프로젝트를 저장할 디렉토리 이름
!git clone https://github.com/genforce/higan.git $CODE_DIR

!mkdir -p higan/models/pretrain/pytorch  # HiGAN 모델 파일을 저장할 디렉토리를 생성
!wget https://www.dropbox.com/s/h1w7ld4hsvte5zf/stylegan_bedroom256_generator.pth?dl=1 -O higan/models/pretrain/pytorch/stylegan_bedroom256_generator.pth --quiet
!wget https://www.dropbox.com/s/hwjyclj749qtp89/order_w.npy?dl=1 -O higan/order_w_1k.npy --quiet


Cloning into 'higan'...
remote: Enumerating objects: 288, done.
remote: Counting objects: 100% (288/288), done.
remote: Compressing objects: 100% (226/226), done.
remote: Total 288 (delta 77), reused 263 (delta 58), pack-reused 0 (from 0)
Receiving objects: 100% (288/288), 16.22 MiB | 4.52 MiB/s, done.
Resolving deltas: 100% (77/77), done.


In [ ]:
# pSp GitHub 저장소 클론
!git clone https://github.com/eladrich/pixel2style2pixel.git
%cd pixel2style2pixel

os.chdir('/content')
!mkdir -p pixel2style2pixel/pretrained_models # pSp Encoder 모델 파일을 저장할 디렉토리를 생성

Cloning into 'pixel2style2pixel'...
remote: Enumerating objects: 418, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 418 (delta 0), reused 2 (delta 0), pack-reused 414 (from 1)
Receiving objects: 100% (418/418), 92.94 MiB | 16.52 MiB/s, done.
Resolving deltas: 100% (147/147), done.
/content/pixel2style2pixel


In [ ]:
# gdown 라이브러리를 설치
!pip install gdown

In [ ]:
# psp_ffhq_encode.pt 다운
import gdown

file_id = "1bMTNWkh5LArlaWSc_wa8VKyq2V42T2z0"
output_path = "pixel2style2pixel/pretrained_models/psp_ffhq_encode.pt"  # 저장 경로
gdown.download(f"https://drive.google.com/uc?id={file_id}", output_path, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1bMTNWkh5LArlaWSc_wa8VKyq2V42T2z0
From (redirected): https://drive.google.com/uc?id=1bMTNWkh5LArlaWSc_wa8VKyq2V42T2z0&confirm=t&uuid=9134d280-2469-4e62-a72d-f31e9d8ce672
To: /content/pixel2style2pixel/pretrained_models/psp_ffhq_encode.pt
100%|██████████| 1.20G/1.20G [00:29<00:00, 41.1MB/s]


'pixel2style2pixel/pretrained_models/psp_ffhq_encode.pt'

In [ ]:
# C++ 확장을 위한 ninja 설치
!apt-get install ninja-build
!pip install ninja

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  ninja-build
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 111 kB of archives.
After this operation, 358 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 ninja-build amd64 1.10.1-1 [111 kB]
Fetched 111 kB in 1s (126 kB/s)
Selecting previously unselected package ninja-build.
(Reading database ... 123633 files and directories currently installed.)
Preparing to unpack .../ninja-build_1.10.1-1_amd64.deb ...
Unpacking ninja-build (1.10.1-1) ...
Setting up ninja-build (1.10.1-1) ...
Processing triggers for man-db (2.10.2-1) ...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 17.8 MB/s eta 0:00:00


In [ ]:
# PyTorch 확장 캐시 디렉토리를 삭제 후 새로 빌드
!rm -rf ~/.cache/torch_extensions

In [ ]:
# CUDA 사용 가능 여부 확인 / 버전
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")

PyTorch version: 2.5.1+cu121
CUDA available: True
CUDA version: 12.1


# Define Utility Functions

In [ ]:
import io
import IPython.display
import cv2
import PIL.Image
import os
import numpy as np
from tqdm import tqdm
import torch

from higan.models.helper import build_generator
from higan.utils.logger import setup_logger
from higan.utils.editor import get_layerwise_manipulation_strength
from higan.utils.editor import manipulate

# 이미지 배열 시각화 함수
def imshow(images, col, viz_size=256):
  """Shows images in one figure."""
  num, height, width, channels = images.shape
  assert num % col == 0
  row = num // col

  fused_image = np.zeros((viz_size * row, viz_size * col, channels), dtype=np.uint8)

  for idx, image in enumerate(images):
    i, j = divmod(idx, col)
    y = i * viz_size
    x = j * viz_size
    if height != viz_size or width != viz_size:
      image = cv2.resize(image, (viz_size, viz_size))
    fused_image[y:y + viz_size, x:x + viz_size] = image

  fused_image = np.asarray(fused_image, dtype=np.uint8)
  data = io.BytesIO()
  PIL.Image.fromarray(fused_image).save(data, 'jpeg')
  im_data = data.getvalue()
  disp = IPython.display.display(IPython.display.Image(im_data))
  return disp

# 모델을 빌드 함수
def build_model(model_name, logger=None):
  """Builds the generator by model name."""
  model = build_generator(model_name, logger=logger)
  return model

# latent_code를 반환해주는 함수
def sample_codes(model, num, seed=0, w1k_code=None):
  """Samples latent codes randomly."""
  np.random.seed(seed)
  if w1k_code is None:
    codes = generator.easy_sample(num)
    latent_codes = model.easy_sample(num=num, latent_space_type='w')
  else:
    latent_codes = w1k_code[np.random.randint(0, w1k_code.shape[0], num)]
  latent_codes = model.easy_synthesize(latent_codes=latent_codes,
                                       latent_space_type='w',
                                       generate_style=False,
                                       generate_image=False)['wp']
  return latent_codes

# 사전 정의된 'w' 잠재 코드 로드
w1k_code = np.load('/content/higan/order_w_1k.npy')

# Build Generator

In [ ]:
indoor_model_name = "stylegan_bedroom"
indoor_model = build_model(indoor_model_name)

# latent code 저장

In [ ]:
import os
import sys
import torch
import numpy as np
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
from argparse import Namespace

# 프로젝트 디렉토리를 PYTHONPATH에 추가
project_dir = '/content/pixel2style2pixel'
sys.path.append(project_dir)

from pixel2style2pixel.models.psp import pSp  # psp 인코더

# -------------------- 장치 설정 --------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -------------------- pSp 인코더 로드 --------------------
# pSp 설정 정의
opts = Namespace(
    checkpoint_path='/content/pixel2style2pixel/pretrained_models/psp_ffhq_encode.pt',  # pSp 인코더 가중치 경로
    device=device,  # GPU/CPU 설정
    output_size=1024,  # 출력 이미지 해상도
    encoder_type='GradualStyleEncoder',  # 인코더 타입
    input_nc=3  # 입력 이미지 채널 수 (RGB: 3)
)

# pSp 초기화
psp_encoder = pSp(opts)

# pSp 가중치 로드
psp_state_dict = torch.load(opts.checkpoint_path, map_location=device)['state_dict']
psp_encoder.load_state_dict(psp_state_dict)
psp_encoder = psp_encoder.to(device)
psp_encoder.eval()  # 평가 모드 설정

# -------------------- 입력 이미지 전처리 --------------------
# 이미지 전처리 함수
def preprocess_image(image_path, device):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # [-1, 1] 범위로 정규화
    ])
    image = Image.open(image_path).convert('RGB')
    return transform(image).unsqueeze(0).to(device)

# 이미지 경로
input_image_path = '/content/room3.png'  # 변환할 입력 이미지 경로
input_image = preprocess_image(input_image_path, device)

# -------------------- pSp 인코더를 통한 잠재 코드 생성 --------------------
with torch.no_grad():
    latent_code = psp_encoder.encoder(input_image)  # 잠재 코드 생성

# -------------------- 잠재 코드 저장 --------------------
latent_code_numpy = latent_code.cpu().numpy()  # GPU 텐서를 CPU로 이동 후 NumPy 배열로 변환
np.save('latent_codes.npy', latent_code_numpy)  # latent space 저장
print("Latent codes saved successfully!")

# -------------------- 저장된 잠재 코드 확인 --------------------
latent_codes_loaded = np.load('latent_codes.npy')
print("Loaded latent codes shape:", latent_codes_loaded.shape)

Loading pSp from checkpoint: /content/pixel2style2pixel/pretrained_models/psp_ffhq_encode.pt


<ipython-input-16-9496b3d47fd3>:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  psp_state_dict = torch.load(opts.checkpoint_path, map_location=device)['state_dict']


Latent codes saved successfully!
Loaded latent codes shape: (1, 18, 512)


# StyleGAN에서 Latent Codes 이미지 생성 시도

In [ ]:
import torch
import numpy as np

attribute_name = 'indoor_lighting'  # @param ['indoor_lighting', 'wood', 'cluttered_space', 'view']
path = f'higan/boundaries/{indoor_model_name}/{attribute_name}_boundary.npy'

# Boundary 파일 로드
try:
    boundary_file = np.load(path, allow_pickle=True).item()
    boundary = boundary_file['boundary']
    manipulate_layers = boundary_file['meta_data']['manipulate_layers']
except ValueError:  # 만약 Boundary 파일 형식이 잘못된 경우
    boundary = np.load(path)
    manipulate_layers = '0-4' if attribute_name == 'view' else '6-11'

# Layer-wise 조작 강도 설정
if attribute_name == 'view':
    strength = [1.0 for _ in range(indoor_model.num_layers)]  # 특정 Layer 조작 강도
else:
    strength = get_layerwise_manipulation_strength(
        indoor_model.num_layers, indoor_model.truncation_psi, indoor_model.truncation_layers
    )

# Latent code 변환 (GPU -> CPU -> NumPy 배열)
if isinstance(latent_code, torch.Tensor):  # 텐서인지 확인
    latent_code_numpy = latent_code.detach().cpu().numpy()  # NumPy 배열로 변환
else:
    latent_code_numpy = latent_code  # 이미 NumPy 배열이라면 그대로 사용

# Boundary도 NumPy 배열로 유지
if isinstance(boundary, torch.Tensor):  # 만약 Boundary가 PyTorch 텐서라면
    boundary_numpy = boundary.cpu().numpy()  # NumPy 배열로 변환
else:
    boundary_numpy = boundary

# 조작 수행
distance = -3  # 조작 거리 (-3.0 ~ 3.0)
indoor_codes = manipulate(
    latent_codes=latent_code_numpy,  # NumPy 배열 사용
    boundary=boundary_numpy,  # NumPy 배열 사용
    start_distance=0,
    end_distance=distance,
    step=2,
    layerwise_manipulation=True,
    num_layers=indoor_model.num_layers,
    manipulate_layers=manipulate_layers,
    is_code_layerwise=True,
    is_boundary_layerwise=False,
    layerwise_manipulation_strength=strength
)

# 불필요한 차원이 있는지 확인하고 제거
if len(indoor_codes.shape) == 4:  # 잘못된 두 번째 차원인 경우
    indoor_codes = indoor_codes[:, 0, :, :]  # 첫 번째 요소만 사용

# 이미지 생성
images = indoor_model.easy_synthesize(indoor_codes, latent_space_type='wp')['image']

# 이미지 시각화
imshow(images, col=1)  # 한 줄에 하나의 이미지 표시

ValueError: Latent codes should be with shape [num, num_layers, *code_shape], where `num_layers` equals to 14, but (1, 18, 512) is received!